In [1]:
import pandas
import numpy
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import ipywidgets as ipy
from datetime import date
import decimal
import seaborn

In [2]:
dataset = pandas.read_csv(r"AER_credit_card_data.csv")
datasetClean = dataset.replace(to_replace = ['yes','no'],value = ['1','0'])

In [3]:
print("First, let's visualize the data that will train our model in a few ways.")

def selectVisual(Visualization):
    if(Visualization == 'Homeowner Approval'):
        g = seaborn.countplot(data=dataset, x = "owner", hue = "card")
        display(g)
        print("The chart below shows that, based on the data, people that do not own homes are more likely to be declined a "
              "new credit card.")
    elif(Visualization == 'Reports Approval'):
        g = seaborn.stripplot(data=dataset, x = "reports", y = "card")
        display(g)
        print("The chart below shows that a cusotomer with more derogatory reports is likely to be declined a new credit card.")
    else:
        g = seaborn.boxenplot(data=dataset, x = "card", y = "income")
        display(g)
        print("The chart below shows a very slight bias towards approving customers with higher incomes.")
ipy.interact(selectVisual, Visualization=['Homeowner Approval', 'Reports Approval', 'Income Approval'])

print(" ")

First, let's visualize the data that will train our model in a few ways.


interactive(children=(Dropdown(description='Visualization', options=('Homeowner Approval', 'Reports Approval',…

In [4]:
labels = datasetClean.iloc[:, 0].values
attributes = datasetClean.iloc[:, 1:12].values

In [5]:
X_train, X_test, y_train, y_test = train_test_split(attributes, labels, test_size=0.25, random_state=0)

In [6]:
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [7]:
classifier = RandomForestClassifier(n_estimators=16, random_state=0)
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

In [8]:
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96        67
           1       1.00      0.98      0.99       263

    accuracy                           0.98       330
   macro avg       0.97      0.99      0.98       330
weighted avg       0.99      0.98      0.99       330

0.9848484848484849


In [9]:
reportsText=ipy.Text(placeholder='Enter the number of reports...', description='Reports:')
print("Enter the number of major derogatory reports for the customer below.")
display(reportsText)

months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']

birthMonDrop = ipy.Dropdown(options=months, description="Birth Month:",)

years = []
thisYear = date.today().year
yearsToInclude = 124
while(yearsToInclude > 0):
    years.append(str(thisYear))
    thisYear -= 1
    yearsToInclude -= 1


birthYearDrop = ipy.Dropdown(options=years, description="Birth Year:")

print("Select the customer's birth month and birth year below.")

ui = ipy.HBox([birthMonDrop, birthYearDrop])

display(ui)

print("Enter the customer's yearly income as a whole dollar value below.")

incomeText=ipy.Text(placeholder='Enter a dollar value...', description='Income:')
display(incomeText)

print("Enter the customer's average monthly credit card expenditure below.")

expenText=ipy.Text(placeholder='Enter average monthly cc expenditure...', description='Expenditure:')
display(expenText)

print("Select whether the customer is a home owner and whether the customer is self employed below.")

homeowner = ''
selfemployed = ''

def myfunction(Homeowner):
    global homeowner
    homeowner = Homeowner
ipy.interact(myfunction, Homeowner=['Yes','No'])

def myfunctiontwo(Selfemployed): 
    global selfemployed
    selfemployed = Selfemployed
ipy.interact(myfunctiontwo, Selfemployed=['Yes','No'])

print("Enter the customer's number of dependents below.")

depenText=ipy.Text(placeholder='Enter the number of dependents...', description='Dependents:')
display(depenText)

print("Enter the number of months the customer has lived at their current address below.")

monthText=ipy.Text(placeholder='Enter months at address...', description='Months:')
display(monthText)

print("Enter the number of major credit cards held by the customer below.")

majorText=ipy.Text(placeholder='Enter number of major CCs...', description='Major Cards:')
display(majorText)

print("Enter the customer's number of active credit accounts below.")

activeText=ipy.Text(placeholder='Enter number of accounts...', description='Accounts:')
display(activeText)

def is_integer(n):
    try:
        float(n)
    except ValueError:
        return False
    else:
        return float(n).is_integer()
    
def is_float(n):
    try:
        float(n)
        return True
    except ValueError:
        return False
    
def getAge(month: str, year: int):
    monNum = months.index(month) + 1
    currentMonth = date.today().month
    currentYear = date.today().year
    if(currentMonth == monNum):
        return decimal.Decimal(currentYear - year)
    if(currentMonth < monNum):
        return decimal.Decimal(currentYear - (year + 1)) + (decimal.Decimal(12 - (monNum - currentMonth)) / decimal.Decimal(12))
    else:
        return decimal.Decimal(currentYear - year) + (decimal.Decimal(currentMonth - monNum) / decimal.Decimal(12))
    


button = ipy.Button(description='Submit Application')
out = ipy.Output()
def on_button_clicked(_):
      with out:
            out.clear_output()
            report = reportsText.value
            if(is_integer(report)):
                if(int(report) < 0):
                    print('\'Reports:\' text box must contain a positive integer.')
                    return
                else:
                    report = int(report)
            else:
                print('\'Reports:\' text box must contain a positive integer.')
                return
            
            age = getAge(birthMonDrop.value, int(birthYearDrop.value))
            
            if(age < 0):
                print('Invalid age. Customer must be born.')
                return
            
            income = incomeText.value
            if(is_integer(income)):
                if(int(income) < 0):
                    print('\'Income:\' text box must contain a positive integer.')
                    return
                else:
                    income = decimal.Decimal(income) / decimal.Decimal(10000)
            else:
                print('\'Income:\' text box must contain a positive integer.')
                return
            
            expend = expenText.value
            if(is_float(expend)):
                if(decimal.Decimal(expend) < 0):
                    print('\'Expenditure:\' text box must contain a positive number.')
                    return
                else:
                    expend = decimal.Decimal(expend)
            else:
                print('\'Expenditure:\' text box must contain a positive number.')
                return
            
            if(income < 1):
                share = 0.0
            else:
                share = (decimal.Decimal(expend) * decimal.Decimal(12)) / (decimal.Decimal(income) * decimal.Decimal(10000))
            
            depend = depenText.value
            if(is_integer(depend)):
                if(int(depend) < 0):
                    print('\'Dependents:\' text box must contain a positive integer.')
                    return
                else:
                    depend = int(depend)
            else:
                print('\'Dependents:\' text box must contain a positive integer.')
                return
            
            month = monthText.value
            if(is_integer(month)):
                if(int(month) < 0):
                    print('\'Months:\' text box must contain a positive integer.')
                    return
                else:
                    month = int(month)
            else:
                print('\'Months:\' text box must contain a positive integer.')
                return
            
            cards = majorText.value
            if(is_integer(cards)):
                if(int(cards) < 0):
                    print('\'Major Cards:\' text box must contain a positive integer.')
                    return
                else:
                    cards = int(cards)
            else:
                print('\'Major Cards:\' text box must contain a positive integer.')
                return
            
            accounts = activeText.value
            if(is_integer(accounts)):
                if(int(accounts) < 0):
                    print('\'Accounts:\' text box must contain a positive integer.')
                    return
                else:
                    accounts = int(accounts)
            else:
                print('\'Accounts:\' text box must contain a positive integer.')
                return
            
            global homeowner
            global selfemployed
            
            if(homeowner == 'Yes' or homeowner == '1'):
                homeowner = '1'
            else:
                homeowner = '0'
                
            if(selfemployed == 'Yes' or selfemployed == '1'):
                selfemployed = '1'
            else:
                selfemployed = '0'
            
            result = [report, age, income, share, expend, homeowner, selfemployed, depend, month, cards, accounts]
            X_test = [result]
            X_test = sc.transform(X_test)
            approval = classifier.predict(X_test)
            if(approval == '1'):
                print('Credit Card Application has been approved.')
            else:
                print('Credit Card Application has been denied.')
button.on_click(on_button_clicked)
ipy.VBox([button,out])


Enter the number of major derogatory reports for the customer below.


Text(value='', description='Reports:', placeholder='Enter the number of reports...')

Select the customer's birth month and birth year below.


Enter the customer's yearly income as a whole dollar value below.


Text(value='', description='Income:', placeholder='Enter a dollar value...')

...

Text(value='', description='Months:', placeholder='Enter months at address...')

Enter the number of major credit cards held by the customer below.


Text(value='', description='Major Cards:', placeholder='Enter number of major CCs...')

Enter the customer's number of active credit accounts below.


Text(value='', description='Accounts:', placeholder='Enter number of accounts...')